In [1]:
from concurrent.futures import ProcessPoolExecutor,as_completed

In [2]:
import statsmodels.api as sm

In [3]:
f = read('fama_macbeth/factors.csv')

In [4]:
dates = sorted(f.date.drop_duplicates().tolist())

In [5]:
window = 48

In [6]:
cal_date = dict()
for i,x in enumerate(dates):
    if i>window:
        cal_date[x] = pd.DataFrame(dates[i-window:i],columns=['date'])

In [7]:
msf = rch('''select permno,toInt32(a.date/100) as date,ret*100-rf as eret, b.*
            from msf a left join factors b on toInt32(a.date/100)=b.date''').dropna()

In [18]:
def reg(g):
#     if len(g)>window*0.8:
        return sm.OLS.from_formula("eret ~ 1 + mktrf + smb",data=g).fit().params

In [14]:
def cal(date):
    if date in cal_date:
        a = msf.merge(cal_date[date]).groupby(['permno']).apply(reg).reset_index()
        a['date'] = date
        return a

In [20]:
len(msf.merge(cal_date[196708])#.groupby(['permno']).apply(reg)#.reset_index()

,permno,date,eret,b.date,mktrf,smb,hml,rmw,cma,rf,umd,liq,bab,ni,resvar,r_me,r_ia,r_roe,r_eg


In [27]:
msf.query('date==197701')

,permno,date,eret,b.date,mktrf,smb,hml,rmw,cma,rf,umd,liq,bab,ni,resvar,r_me,r_ia,r_roe,r_eg
42851,79231,197701,-0.36,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
47306,79250,197701,-0.36,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
49765,79266,197701,6.91,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
50544,79274,197701,-1.32,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
54386,79311,197701,-10.89,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4691254,48856,197701,3.49,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
4691530,48864,197701,58.82,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
4691675,48872,197701,-7.70,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65
4691792,48873,197701,-0.36,197701,-4.05,5.90,4.27,-0.51,1.97,0.36,0.04,-0.03,4.07,2.83,-5.54,6.05,1.00,-2.15,1.65


In [23]:
cal_date[196708]

,date
0,196308
1,196309
2,196310
3,196311
4,196312
5,196401
6,196402
7,196403
8,196404
9,196405


In [ ]:
res = []
with ProcessPoolExecutor() as p:
    for date in dates:
        res.append(p.submit(cal,date))
    res = as_completed(res)
    x = pd.concat([a.result() for a in res if a.result() is not None])

In [ ]:
x